# Let's evaluate the measurement noise

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
from tqdm.auto import tqdm
import numpy as np
from matplotlib.lines import Line2D


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
base_dir = os.path.dirname(parent_dir)
data_dir = base_dir + "/data/octomag_data/split_dataset/"
src_dir = base_dir + "/src"
plot_dir = cwd + "/plots/"

sys.path.insert(0, src_dir)

from calibration import MPEM, ActuationNet
from paper import latex_utils

## Load data

In [ ]:
df = pd.read_pickle(data_dir + "/test_data.pkl")

## Load model

In [ ]:
model_path = base_dir + "/mpem/optimized_parameters/" + "optimized_dipole_5.yaml"
model = MPEM(model_path)

## Helpful funcs

In [ ]:
def skew_sym(vector):
    """Create a skew-symmetric matrix from a 3D vector."""
    x, y, z = vector
    return np.array([[0, -z, y],
                     [z, 0, -x],
                     [-y, x, 0]])

def get_pos_jacobian_and_field(pos, currents, model):
    """Compute the Jacobian of the magnetic field w.r.t. position using autograd."""
    pos = np.array(pos, )
    currents = np.array(currents)

    field, jacobian_pos = model.get_field_and_grad9(pos, currents)

    return field, jacobian_pos

def build_sensitivity_jacobians(positions, currents, model):

    Js_pos = []
    Js_angle = []
    
    for position, current in tqdm(zip(positions, currents), total=len(positions), desc="Building sensitivity Jacobians"):
        B_pred, J_pos = get_pos_jacobian_and_field(position, current, model)
        Js_pos.append(J_pos)
        Js_angle.append(skew_sym(B_pred))

    J_pos = np.array(Js_pos)
    J_angle = np.array(Js_angle)

    return J_pos, J_angle

import numpy as np

def compute_rse_total(J_pos, J_angle, iso_pos_std, iso_angle_std):
    # Input isotropic covariances
    cov_pos   = (iso_pos_std   ** 2) * np.eye(3)
    cov_angle = (iso_angle_std ** 2) * np.eye(3)

    # Ensure batched shape (N,3,3)
    J_pos = np.asarray(J_pos)
    J_angle = np.asarray(J_angle)
    if J_pos.ndim == 2:
        J_pos = J_pos[None, ...]
    if J_angle.ndim == 2:
        J_angle = J_angle[None, ...]

    # Cov_total[i] = J_pos[i] cov_pos J_pos[i]^T + J_angle[i] cov_angle J_angle[i]^T
    JT_pos = np.transpose(J_pos, (0, 2, 1))
    JT_ang = np.transpose(J_angle, (0, 2, 1))

    cov_total = (J_pos @ cov_pos @ JT_pos) + (J_angle @ cov_angle @ JT_ang)

    # RSE per sample and RMSE over samples
    tr = np.trace(cov_total, axis1=1, axis2=2)
    tr = np.maximum(tr, 0.0)  # numerical guard

    rse = np.sqrt(tr)              # (N,)
    rmse = np.sqrt(np.mean(tr))    # scalar

    return rse, rmse

## Define the angle and position deviations to test (deg and mm)

In [ ]:
angle_stds_deg_min, angle_stds_deg_min_max = 0, 2.0 
pos_stds_mm_min, pos_stds_mm_max = 0, 2.0

## Plot heat map of the rmse

In [ ]:
# Build Jacobians
positions = df[["x", "y", "z"]].to_numpy()
currents  = df[[c for c in df.columns if c.startswith("em_")]].to_numpy()

J_pos, J_angle = build_sensitivity_jacobians(positions, currents, model)

# Define grid (deg, mm)
num_divs= 100
angle_grid_deg = np.linspace(angle_stds_deg_min, angle_stds_deg_min_max, num_divs)
pos_grid_mm    = np.linspace(pos_stds_mm_min, pos_stds_mm_max, num_divs)

# Compute RMSE over grid (convert to SI: rad, m)
rmse_grid = np.empty((num_divs, num_divs), dtype=float)  # rows=pos, cols=angle

for i, pos_std_mm in enumerate(tqdm(pos_grid_mm, desc="RMSE grid (pos)")):
    pos_std_m = pos_std_mm / 1000.0
    for j, ang_std_deg in enumerate(angle_grid_deg):
        ang_std_rad = np.deg2rad(ang_std_deg)

        _, rmse = compute_rse_total(J_pos, J_angle, pos_std_m, ang_std_rad)
        rmse_grid[i, j] = rmse

In [ ]:
def plot_rmse_heatmap_latex(
    rmse_grid,
    angle_grid_deg,
    pos_grid_mm,
    *,
    figsize=(3.65, 2.85),

    # display downsample only (keeps file light)
    step=2,

    tick_step_deg=0.5,
    tick_step_mm=0.5,

    # labels / title 
    xlabel=r"Orientation $\sigma_{\theta} (\deg)$",
    ylabel=r"Position $\sigma_p$ (mm)",
    cbar_label=r"$\mathrm{RMSE}_{\mathrm{floor}}\ (\mathrm{mT})$",
    suptitle=None,

    # --- iso-RMSE contour line ---
    iso_rmse=0.5,                       # mT
    iso_rmse_color="red",
    iso_rmse_label=None,
    iso_rmse_lw=1.2,
    iso_rmse_ls="-",
    iso_rmse_alpha=0.95,

    # --- legend placement (below x-axis) ---
    legend_below=True,
    legend_y=0.05,
    legend_fontsize=None,

    # latex rc context
    latex_params=None,
    usetex=True,

    # appearance
    cmap="viridis",
    vmin=None,
    vmax=None,

    # layout margins
    left=0.18,
    right=0.98,
    bottom=0.30,
    top=0.92,

    # store
    store_to = "rmse_heatmap.pdf"
):
    

    rmse_grid = np.asarray(rmse_grid)
    angle_grid_deg = np.asarray(angle_grid_deg)
    pos_grid_mm = np.asarray(pos_grid_mm)

    if rmse_grid.ndim != 2:
        raise ValueError(f"rmse_grid must be 2D, got shape {rmse_grid.shape}")
    if rmse_grid.shape != (len(pos_grid_mm), len(angle_grid_deg)):
        raise ValueError(
            f"Shape mismatch: rmse_grid {rmse_grid.shape} vs "
            f"pos_grid_mm {len(pos_grid_mm)} and angle_grid_deg {len(angle_grid_deg)}"
        )

    step = max(int(step), 1)
    rmse_disp = rmse_grid[::step, ::step]
    angle_disp = angle_grid_deg[::step]
    pos_disp = pos_grid_mm[::step]

    # physical extent for imshow
    a0, a1 = float(angle_disp.min()), float(angle_disp.max())
    p0, p1 = float(pos_disp.min()), float(pos_disp.max())
    extent = [a0, a1, p0, p1]

    # ticks at fixed resolution (clipped to range)
    def _ticks(lo, hi, step_):
        step_ = float(step_)
        lo_t = np.ceil(lo / step_) * step_
        hi_t = np.floor(hi / step_) * step_
        if hi_t < lo_t:
            return np.array([lo, hi])
        return np.round(np.arange(lo_t, hi_t + 0.5 * step_, step_), 6)

    xticks = _ticks(a0, a1, tick_step_deg)
    yticks = _ticks(p0, p1, tick_step_mm)

    latex_params = {} if latex_params is None else dict(latex_params)
    latex_params.setdefault("figsize", figsize)
    latex_params.setdefault("usetex", usetex)

    with latex_utils.rc_context_latex(**latex_params):
        fig, ax = plt.subplots(figsize=figsize, constrained_layout=False)
        fig.subplots_adjust(left=left, right=right, bottom=bottom, top=top)

        im = ax.imshow(
            rmse_disp,
            origin="lower",
            aspect="auto",
            extent=extent,
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            interpolation="nearest",
        )

        ax.set_xticks(xticks)
        ax.set_yticks(yticks)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)

        if suptitle:
            fig.suptitle(suptitle)

        cbar = fig.colorbar(im, ax=ax, pad=0.02)
        cbar.set_label(cbar_label)

        # --- iso-RMSE contour line ---
        did_draw = False
        if np.isfinite(iso_rmse) and (rmse_disp.min() <= iso_rmse <= rmse_disp.max()):
            A, P = np.meshgrid(angle_disp, pos_disp)
            ax.contour(
                A, P, rmse_disp,
                levels=[float(iso_rmse)],
                colors=[iso_rmse_color],
                linewidths=iso_rmse_lw,
                linestyles=iso_rmse_ls,
                alpha=iso_rmse_alpha,
            )
            did_draw = True

        # legend below the x-axis
        if legend_below and did_draw:
            if iso_rmse_label is None:
                iso_rmse_label = rf"Iso-RMSE: {iso_rmse:g}\,mT"
            handle = Line2D([0], [0], color=iso_rmse_color, lw=iso_rmse_lw, ls=iso_rmse_ls, alpha=iso_rmse_alpha)
            fig.legend(
                handles=[handle],
                labels=[iso_rmse_label],
                loc="lower center",
                bbox_to_anchor=(0.5, legend_y),
                frameon=False,
                ncol=1,
                fontsize=legend_fontsize,
            )

        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(store_to), exist_ok=True)

        fig.savefig(store_to, bbox_inches="tight", pad_inches=0)

        plt.show()

    return fig, ax

In [ ]:
fig, ax = plot_rmse_heatmap_latex(
    rmse_grid,
    angle_grid_deg,
    pos_grid_mm,
    step=2,
    figsize=(3.65, 2.85),
    tick_step_deg=0.5,
    tick_step_mm=0.5,
    latex_params=latex_utils.latex_prms_singlecol,
    usetex=True,
    store_to=plot_dir + "rmse_heatmap.pdf",
)